In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Initial Data Manipulation

In [22]:

def add_truck_sequences(file_path):
    df = pd.read_csv(file_path)

    df['inbound_date'] = pd.to_datetime(df['inbound_date'])
    df['planned_arrival_central_time'] = pd.to_datetime(df['planned_arrival_central_time']).dt.time
    df['actual_arrival_central_time'] = pd.to_datetime(df['actual_arrival_central_time']).dt.time

    df['planned_arrival_datetime'] = df.apply(lambda row: pd.Timestamp.combine(row['inbound_date'], row['planned_arrival_central_time']), axis =1)
    df['actual_arrival_datetime'] = df.apply(lambda row: pd.Timestamp.combine(row['inbound_date'], row['actual_arrival_central_time']), axis =1)

    def adjust_date(row, time_column):
        if row[time_column].time() >= pd.Timestamp('12:00').time():
            return row['inbound_date'] + pd.Timedelta(days = 1)
        else:
            return row['inbound_date'] 
    
    df['adjusted_planned_date'] = df.apply(adjust_date, axis = 1, time_column = 'planned_arrival_datetime')
    df['adjusted_actual_date'] = df.apply(adjust_date, axis = 1, time_column = 'actual_arrival_datetime')

    df = df.sort_values(by = ['sortation_center_id', 'adjusted_planned_date', 'planned_arrival_datetime'])
    df['[planned_truck_sequence'] = df.groupby(['sortation_center_id', 'adjusted_planned_date']).cumcount()+1

    df = df.sort_values(by = ['sortation_center_id', 'adjusted_actual_date', 'actual_arrival_datetime'])
    df['actual_truck_sequence'] = df.groupby(['sortation_center_id', 'adjusted_actual_date']).cumcount()+1

    df = df.drop(columns = ['adjusted_planned_date', 'adjusted_actual_date'])

    return df

def filter_time_range(file_path, output_path):
    df = pd.read_csv(file_path)

    df['Prediction_Time'] = pd.to_datetime(df['Prediction_Time'])

    filtered_df = df[(df['Prediction_Time'].dt.time>= pd.Timestamp('12:00').time()) &
                    (df['Prediction_Time'].dt.time < pd.Timestamp('13:00').time())]
    
    filtered_df.to_csv(output_path, index = False)

In [24]:
file_path = 'linehaul_all.csv'
df_with_sequences = add_truck_sequences(file_path)
df_with_sequences.to_csv('linhaul_all_sequence.csv', index = False)

C:\Users\Z00F0LC\AppData\Local\Temp\1\ipykernel_25668\2536436517.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['planned_arrival_central_time'] = pd.to_datetime(df['planned_arrival_central_time']).dt.time
C:\Users\Z00F0LC\AppData\Local\Temp\1\ipykernel_25668\2536436517.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['actual_arrival_central_time'] = pd.to_datetime(df['actual_arrival_central_time']).dt.time


In [15]:
file_path = 'Raft Predict.csv'
output_path = 'Raft_Predict_filtered.csv'

filter_time_range(file_path, output_path)

# Random Forrest Implementation

In [17]:




# Load the CSV file into a DataFrame
df = pd.read_csv('Inbound_predict_data.csv')

# one hot encode day of week

df_onehot = pd.get_dummies(df, columns = ['Day of Week'])


# Feature selection
features = ['RAFT_predicted_inbound_remaining_pkg_count', 
            'RAFT_known_shipped_pkg_count', 
            'RAFT_predicted_inbound_pkg_count', 
            'RAFT_predicted_carryover_pkg_count', 
            'RAFT_capped_predicted_inbound_pkg_count', 
            'RAFT_predicted_total_handoff_pkg_count', 
            'Day of Week_Sunday',
            'Day of Week_Monday',
            'Day of Week_Tuesday',
            'Day of Week_Wednesday',
            'Day of Week_Thursday',
            'Day of Week_Friday',
            'Day of Week_Saturday',
            'Promotion', 
            'TMAX', 
            'TMIN', 
            'AWND',
            'PRCP', 
            'SNOW'
            ]

# Convert date columns to datetime
df_onehot['Prediction_For_Date'] = pd.to_datetime(df['Prediction_For_Date'])

# Extract additional features from date columns
#df['day'] = df['Prediction_For_Date'].dt.day
#df['month'] = df['Prediction_For_Date'].dt.month

# Update features list with new date features
#features.extend(['day', 'month'])

# Split the data into training and testing sets
X = df_onehot[features]
y_package_count = df_onehot['Total Packages Received']
#y_arrival_time = df['actual_arrival_time']

X_train, X_test, y_package_train, y_package_test = train_test_split(X, y_package_count, test_size=0.2, random_state=42)
#X_train, X_test, y_arrival_train, y_arrival_test = train_test_split(X, y_arrival_time, test_size=0.2, random_state=42)

# Train the model for actual package count prediction
package_model = RandomForestRegressor(n_estimators=100, random_state=42)
package_model.fit(X_train, y_package_train)

# Predict and evaluate the model for actual package count
y_package_pred = package_model.predict(X_test)
r2 = r2_score(y_package_test, y_package_pred)
package_mae = mean_absolute_error(y_package_test, y_package_pred)
print(f'Package Count Prediction MAE: {package_mae}')
print(f'Package Count R2: {r2}')

# Train the model for actual arrival time prediction
#arrival_model = RandomForestRegressor(n_estimators=100, random_state=42)
#arrival_model.fit(X_train, y_arrival_train)

# Predict and evaluate the model for actual arrival time
#y_arrival_pred = arrival_model.predict(X_test)
#arrival_mae = mean_absolute_error(y_arrival_test, y_arrival_pred)
#print(f'Arrival Time Prediction MAE: {arrival_mae}')


Package Count Prediction MAE: 1871.8645454545451
Package Count R2: 0.540807046918459


# Linear Regression Implementation

In [69]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
threshold = 0.95

correlation_matrix = pd.DataFrame(X_scaled,columns=X.columns).corr()

#correlation_matrix_style = correlation_matrix.style.background_gradient(cmap='coolwarm')
print("Correlation Matrix:")
correlation_matrix

Correlation Matrix:


,3_day_average_total_packages,yesterday_total_packages,RAFT_predicted_inbound_remaining_pkg_count,RAFT_known_shipped_pkg_count,RAFT_predicted_inbound_pkg_count,RAFT_predicted_carryover_pkg_count,RAFT_capped_predicted_inbound_pkg_count,RAFT_predicted_total_handoff_pkg_count,Day of Week_Sunday,Day of Week_Monday,...,Day of Week_Wednesday,Day of Week_Thursday,Day of Week_Friday,Day of Week_Saturday,Promotion,TMAX,TMIN,AWND,PRCP,SNOW
3_day_average_total_packages,1.000000,0.853611,0.563357,0.572813,0.655859,0.251961,0.631154,0.648091,0.061471,0.055881,...,-0.040429,-0.051906,-0.041706,0.018360,0.652797,0.013664,-0.015495,-0.123413,-0.005735,0.174572
yesterday_total_packages,0.853611,1.000000,0.463648,0.519512,0.565183,0.152976,0.527290,0.527084,0.034336,-0.027784,...,-0.049745,-0.055092,0.008380,0.120274,0.621943,0.023749,0.003803,-0.087607,-0.001218,0.159355
RAFT_predicted_inbound_remaining_pkg_count,0.563357,0.463648,1.000000,0.494684,0.888345,0.252139,0.844668,0.846138,-0.062933,0.216497,...,0.014433,-0.071626,-0.158083,-0.044826,0.446824,-0.034359,-0.045129,-0.118613,0.005706,0.101394
RAFT_known_shipped_pkg_count,0.572813,0.519512,0.494684,1.000000,0.838509,0.282530,0.739303,0.756024,-0.021526,0.102720,...,0.057199,-0.052466,-0.083088,-0.178866,0.468501,0.034685,0.038642,-0.100279,-0.092512,0.132139
RAFT_predicted_inbound_pkg_count,0.655859,0.565183,0.888345,0.838509,1.000000,0.307361,0.920200,0.929957,-0.050831,0.190011,...,0.039271,-0.072628,-0.143014,-0.122609,0.527682,-0.003217,-0.007879,-0.127350,-0.045301,0.133388
RAFT_predicted_carryover_pkg_count,0.251961,0.152976,0.252139,0.282530,0.307361,1.000000,0.168426,0.405605,-0.119866,0.100852,...,-0.014151,-0.013957,-0.082192,-0.087972,0.032873,0.070953,0.080858,0.009532,-0.098828,-0.065375
RAFT_capped_predicted_inbound_pkg_count,0.631154,0.527290,0.844668,0.739303,0.920200,0.168426,1.000000,0.969305,-0.103085,0.142408,...,0.073776,-0.039732,-0.116149,-0.093306,0.532276,0.089656,0.041157,-0.171732,-0.051328,-0.031276
RAFT_predicted_total_handoff_pkg_count,0.648091,0.527084,0.846138,0.756024,0.929957,0.405605,0.969305,1.000000,-0.125482,0.157184,...,0.064897,-0.040303,-0.128217,-0.108500,0.501772,0.100856,0.058344,-0.156844,-0.072237,-0.045338
Day of Week_Sunday,0.061471,0.034336,-0.062933,-0.021526,-0.050831,-0.119866,-0.103085,-0.125482,1.000000,-0.173913,...,-0.169638,-0.169638,-0.165315,-0.169638,-0.059222,-0.006001,-0.019597,-0.061821,-0.035747,-0.038295
Day of Week_Monday,0.055881,-0.027784,0.216497,0.102720,0.190011,0.100852,0.142408,0.157184,-0.173913,1.000000,...,-0.169638,-0.169638,-0.165315,-0.169638,0.004738,-0.018669,0.001741,-0.132771,0.119846,0.137629


In [68]:

# Load the CSV file into a DataFrame
df = pd.read_csv('Inbound_predict_data.csv')

# one hot encode day of week

df_onehot = pd.get_dummies(df, columns = ['Day of Week'])
df_onehot = df_onehot.iloc[3:]

# Feature selection
features = ['3_day_average_total_packages',   
            'yesterday_total_packages',
            'RAFT_predicted_inbound_remaining_pkg_count', 
            'RAFT_known_shipped_pkg_count', 
            'RAFT_predicted_inbound_pkg_count', 
            'RAFT_predicted_carryover_pkg_count', 
            'RAFT_capped_predicted_inbound_pkg_count', 
            'RAFT_predicted_total_handoff_pkg_count', 
            'Day of Week_Sunday',
            'Day of Week_Monday',
            'Day of Week_Tuesday',
            'Day of Week_Wednesday',
            'Day of Week_Thursday',
            'Day of Week_Friday',
            'Day of Week_Saturday',
            'Promotion', 
            'TMAX', 
            'TMIN', 
            'AWND',
            'PRCP', 
            'SNOW'  
            ]


# Convert date columns to datetime
df_onehot['Prediction_For_Date'] = pd.to_datetime(df['Prediction_For_Date'])

# Extract additional features from date columns
#df['day'] = df['Prediction_For_Date'].dt.day
#df['month'] = df['Prediction_For_Date'].dt.month

# Update features list with new date features
#features.extend(['day', 'month'])

# Split the data into training and testing sets
X = df_onehot[features]
y_package_count = df_onehot['Total Packages Received']
#y_arrival_time = df['actual_arrival_time']

X_train, X_test, y_package_train, y_package_test = train_test_split(X, y_package_count, test_size=0.2, random_state=42)
#X_train, X_test, y_arrival_train, y_arrival_test = train_test_split(X, y_arrival_time, test_size=0.2, random_state=42)

# Train the model for actual package count prediction
package_model = LinearRegression()
package_model.fit(X_train, y_package_train)

# Predict and evaluate the model for actual package count
y_package_pred = package_model.predict(X_test)


r2 = r2_score(y_package_test, y_package_pred)
package_mae = mean_absolute_error(y_package_test, y_package_pred)
print(f'Package Count Prediction MAE: {package_mae}')
print(f'Package Count R2: {r2}')

feature_weights = package_model.coef_
print(f'Coefficients: {feature_weights}')
# Train the model for actual arrival time prediction
#arrival_model = RandomForestRegressor(n_estimators=100, random_state=42)
#arrival_model.fit(X_train, y_arrival_train)

# Predict and evaluate the model for actual arrival time
#y_arrival_pred = arrival_model.predict(X_test)
#arrival_mae = mean_absolute_error(y_arrival_test, y_arrival_pred)
#print(f'Arrival Time Prediction MAE: {arrival_mae}')

Package Count Prediction MAE: 1458.2746714884702
Package Count R2: 0.6683171507690976
Coefficients: [-4.83068283e-01  9.52533904e-01  4.51652362e-01 -1.83238595e-01
  2.68413767e-01 -1.51496038e+02 -1.51421043e+02  1.51308587e+02
  5.93456513e+00 -6.60526077e+02  1.60424162e+02 -9.85847284e+01
  6.22827437e+01  1.04142815e+03 -5.10958813e+02  1.70184980e+03
  4.26187524e+00 -8.29886820e+00  1.91205092e+01  3.32256039e+02
 -1.78378878e+02]


In [1]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
    --------------------------------------- 1.6/124.9 MB 8.4 MB/s eta 0:00:15
   - -------------------------------------- 5.8/124.9 MB 14.7 MB/s eta 0:00:09
   --- ------------------------------------ 9.4/124.9 MB 15.9 MB/s eta 0:00:08
   ---- ----------------------------------- 13.9/124.9 MB 17.1 MB/s eta 0:00:07
   ------ --------------------------------- 19.1/124.9 MB 18.6 MB/s eta 0:00:06
   ------- -------------------------------- 23.3/124.9 MB 18.7 MB/s eta 0:00:06
   --------- ------------------------------ 28.8/124.9 MB 19.9 MB/s eta 0:00:05
   ---------- ----------------------------- 33.6/124.9 MB 20.1 MB/s eta 0:00:05
   ------------ --------------------------- 38.0/124.9 MB 20.3 MB/s eta 0:00:05
   ------------- -------------------------- 42.2/124.9 MB 20.2 MB/s eta 0:00:05
   --------------- ------------------------ 47.2/124.9 MB 20.7 MB/s eta 0:00:04
   ---------------- ----------------------- 52.7/124.

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# Load the CSV file into a DataFrame
df = pd.read_csv('Inbound_predict_data.csv')

# one hot encode day of week

df_onehot = pd.get_dummies(df, columns = ['Day of Week'])
df_onehot = df_onehot.iloc[3:]

# Define features and target
features = ['yesterday_total_packages',
            'RAFT_known_shipped_pkg_count',
            'RAFT_predicted_carryover_pkg_count',
            'RAFT_predicted_total_handoff_pkg_count',
            'Day of Week_Sunday',
            'Day of Week_Monday',
            'Day of Week_Tuesday',
            'Day of Week_Wednesday',
            'Day of Week_Thursday',
            'Day of Week_Friday',
            'Day of Week_Saturday',
            'Promotion',
            'TMAX',
            'TMIN',
            'AWND',
            'PRCP',
            'SNOW']
target = 'Total Packages Received'

# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate MAE and R2
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {r2}')